# C1 W3 Group 8 - Task 5

In [ ]:
from tqdm import tqdm
from PIL import Image
import pickle
from src.background import get_painting_masks, crop_image_by_mask
from src.data import GT_QSD2_W3_LIST
from src.denoising import denoise_image
from src.paths import QSD2_W3_PATH, WEEK_3_RESULTS_PATH, BBDD_PATH, QST1_W3_PATH, QST2_W3_PATH
from src.retrieval import find_top_k_similar_images
from src.utils import show_images

In [ ]:
query_t1_image_PIL_list = [Image.open(query_img_path) for query_img_path in sorted(QST1_W3_PATH.glob("*.jpg"))]  # Load once
query_t2_image_PIL_list = [Image.open(query_img_path) for query_img_path in sorted(QST2_W3_PATH.glob("*.jpg"))]  # Load once
database_image_PIL_list = [Image.open(db_img_path) for db_img_path in sorted(BBDD_PATH.glob("*.jpg"))]  # Load once

###  Denoising

In [ ]:
# Apply median filter
query_t1_image_PIL_list = [denoise_image(image) for image in query_t1_image_PIL_list]
query_t2_image_PIL_list = [denoise_image(image) for image in query_t2_image_PIL_list]

### T1 : Retrieval

In [ ]:
retrieved_images_t1 = find_top_k_similar_images(query_t1_image_PIL_list, database_image_PIL_list, k=10)

In [ ]:
method1_qst1_path = WEEK_3_RESULTS_PATH / "Task_4" / "m1_qst1.pkl"
with open(method1_qst1_path, 'wb') as f:
    pickle.dump(retrieved_images_t1, f)


### T2 : Background removal

In [ ]:
# Get painting masks for T2
painting_masks_list_t2 = []
pbar = tqdm(zip(sorted(QST2_W3_PATH.glob("*.jpg")), query_t2_image_PIL_list))
for name, image in pbar:
    pbar.set_description(f"Processing {name.stem}")
    painting_masks = get_painting_masks(image)    
    painting_masks_list_t2.append(painting_masks)

In [ ]:
method1_qst2_masks_path = WEEK_3_RESULTS_PATH / "Task_4" / "masks_qst2"
import numpy as np
for idx, mask in enumerate(painting_masks_list_t2):
    image_mask = Image.fromarray(mask.astype(np.uint8))
    image_mask.save(method1_qst2_masks_path / f"{idx:05}.png")
    

In [ ]:
cropped_query_image_list_t2 = []
pbar = tqdm(zip(sorted(QST2_W3_PATH.glob("*.jpg")), query_t2_image_PIL_list, painting_masks_list_t2))
for name, image, predicted_mask in pbar:
    pbar.set_description(f"Splitting {name.stem}")
    image_list = crop_image_by_mask(image, predicted_mask)
    cropped_query_image_list_t2.append(image_list)

### T2 : Retrieval

In [ ]:
retrieved_images_t2 = find_top_k_similar_images(cropped_query_image_list_t2, database_image_PIL_list, k=10)
print(retrieved_images_t2)
method1_qst2_path = WEEK_3_RESULTS_PATH / "Task_4" / "m1_qst2.pkl"
with open(method1_qst2_path, 'wb') as f:
    pickle.dump(retrieved_images_t2, f)

In [ ]:
for paintings_image, retrieved_paintings in zip(cropped_query_image_list_t2, retrieved_images_t2):
    print(retrieved_paintings)
    show_images(paintings_image)